# PURSUE Scikit-HEP Session Lecture Notebook (Part 1)

## Introduction

- Uproot is a Scikit-HEP package which offers tools to read and write `.root` files without the use of ROOT.
- In this session, we will be learning about this useful package by going through the [HSF Uproot Tutorial](https://masonproffitt.github.io/uproot-tutorial/).
- This notebook is meant to summarize the material found in the afformentioned tutorial, while expanding on some points not emphasized there, and can be used as a quick reference or to follow along during the session. 
- For a more complete description of Uproot and other Scikit-HEP packages, please visit the [Scikit-HEP Project Website](https://scikit-hep.org/)

## ROOT?
- Root is a specialized data format designed to store large amounts of data in a structured way.
- They can contain a variety of data types such as histograms.
- One of the key features of a root file is the TTree structure in which each entry represents an event and each branch various pieces of data associated with said event.

## Arrays
- When running numerical computations involving a collection of numbers, the usual object of choise is the numpy array. With these we can perform fast computations (Numpy is a compiled library).

In [ ]:
import numpy as np

arr1 = np.array(
    [1, 2, 3, 4, 5]
)
arr2 = np.array(
    [5, 4, 3, 2, 1]
)

- Numpy allows us to do operations between arrays instead of element by element. This type of array-wise operations are called vectorized operations and they are fundamental to a programming paradigm known as array-oriented programming. This is in constrast to the typical approach of iterating over a whole collection of objects, which can be rather slow procedure, as illustrated below.

<img src="./assets/vectorized.png" alt="vectorized" style="width: 400px"/>

In [ ]:
%%timeit
# Slow way
sumrslt = []
for i, j in zip(arr1, arr2):
    sumrslt.append(i + j)
sumarr = np.array(sumrslt)

In [ ]:
%%timeit
# Faster way!
rsltarr = arr1 + arr2 # It also looks much nicer!

- Vectorized operations such as the ones allowed by numpy are not only faster, but, as you can see, have a more elegant syntax.
- Numpy allows us to perform a slew of vectorized operations.

In [ ]:
# Mathematical operations
print("{} - {} = {}".format(arr1, arr2, arr1 - arr2))
print("{} / {} = {}".format(arr1, arr2, arr1 / arr2))
print("{} * {} = {}".format(arr1, arr2, arr1 * arr2))
print("{} % {} = {}".format(arr1, arr2, arr1 % arr2))
print()

# Comparison operations
eq_comp = arr1 == arr2
print("{} == {} = {}".format(arr1, arr2, eq_comp))

ineq_comp = arr1 != arr2
print("{} != {} = {}".format(arr1, arr2, ineq_comp))

print("{} < {} = {}".format(arr1, arr2, arr1 < arr2))
print()

# Logical operations
print("NOT: ~{} = {}".format(eq_comp, ~eq_comp))
print("AND: {} & {} = {}".format(ineq_comp, eq_comp, ineq_comp & eq_comp))
print("OR: {} | {} = {}".format(eq_comp, ineq_comp, eq_comp | ineq_comp))


- Furthermore, with numpy, we can select construct higher dimensional object such as matrices.

In [ ]:
# Convolution example with numpy
from scipy.signal import convolve2d
gradient_kernel = np.array([
                            [-1, 0, 1],
                            [-1, 0, 1],
                            [-1, 0, 1]
                        ])
picture = np.array([
            [0, 255, 0],
            [0, 255, 0],
            [0, 255, 0]
        ])

# Performing a convolution on our "image"
convolve2d(picture, gradient_kernel, mode="same")

## Jagged Arrays
- One of the limitations of numpy array is that if we construct, say, a matrix, it must have a homogeneous shape.


In [ ]:
np.array([
    [1, 2, 3],
    [1],
    [5, 7, 8, 100]
])

- In HEP, you might events with different number of the same particle. This means that the length of object collections (e.g. $p_T$ of detected muons) is not constant. In order to deal with this type of data, we use jagged arrays through the Awakrd library, which builds upon numpy by giving us the ability to have numpy-like arrays, but which can have a non-homogenous shape.
- Awkward Array is a library for using nested, variable-sized data with a Numpy-like syntax.

In [ ]:
import awkward as ak

awk_arr = ak.Array([
    [1, 2, 3],
    [1],
    [5, 7, 8, 100]
])

type(awk_arr)

In [ ]:
ak.sum(awk_arr, axis=1)

- In order to see truly understand how powerful this new type of array is, we will use some real physics data from CMS. In roder to do this, we will need to introduce one of the fundamental tools of the Sciki-HEP tool-kit: Uproot.

## Opening ROOT files with Uproot
- We will be using a `root` file which contains real CMS data of pp collisions in 2012.
- Root files contents are structured in a way reminiscent of a small filesystem with directories and nested directories.
- <>

<img src="./assets/roottree.png" alt="roottree" style="width: 400px"/>

In [ ]:
# Run this cell to import Uproot
import uproot

In [ ]:
# Run this cell to download the data if you haven't already.
! wget https://github.com/masonproffitt/uproot-tutorial-notebooks/raw/master/uproot-tutorial-file.root .

- To open the file, we can the use the `open` function provided by Uproot.

In [ ]:
# Opening the root file
file = uproot.open("./uproot-tutorial-file.root")
file

- We can check the name of the contents at the upper-most level of this "directory" structure.
- Notice that the syntax used here is similar to that of dictionaries.

In [ ]:
# Checking the name of the contents
file.keys()

In [ ]:
# We can also check the type of each of the contents
file.classnames()

- A TTree is is a class which stores data in a way similar to Pandas DataFrames. Each "column" is a TBranch and it can contain objects of any C++ type.
- In the cell below, notice that each the `Events` TTree has 6 branches.

In [ ]:
# We can access the TTree like this
file["Events"]

- We can access the contents of this TTree the following way. When we do, we can see the name of each branch, which describes what content each branch has. 

In [ ]:
tree = file["Events"]
tree.keys()

- Note that you can access branches in a tree more directly with the following syntax. You can use whatever syntax you prefer.

In [ ]:
file["Events;1/nMuon"]

- To emphasize the point about TTrees having a similar structure to Pandas DataFrames, notice that the data can be transformed into a DataFrame relatively easily.

In [ ]:
import pandas as pd

tree.arrays(library="pd", entry_stop=10)

- Of course, this is not a good way to handle this sort of data. This is just for pedagogical purposes.
- In order to convert the data in one of the branches to an array, we would do something like the following.

In [ ]:
branches = tree.arrays()
branches

- Notice that the data type of `branches` is an Akward array (we have come full circle!). As you can deduce, one of the fundamental dependencies of Uproot is Awkward, given that most of the data that we deal with in HEP can be organized into jagged arrays

In [ ]:
type(branches)

- Moreover, if we take a look at the type of the elements of this jagged array, we can find a new type of class provided by Awkward: Record. Records are the Awkward equivalent of Python dictionaries.

In [ ]:
type(branches[0])

In [ ]:
branches[0]

- When we have Records inside Awkward Arrays, we can do something pretty useful. Suppose we want the muon $p_T$ for all of the events. Instead of doing this.

In [ ]:
#%%timeit
# Uncomment the magic above and you will see that this way is slower!
branches[:]["Muon_eta"]

- We can instead do this.

In [ ]:
#%%timeit
branches["Muon_pt"]

- Or suppose I want the number of muons in each event. Again, we can just do this...

In [ ]:
# Array
branches["nMuon"]

- This set-up where we have Records inside an Awkward array, allows much flexibility when it comes to selection and slicing. Here are some examples

In [ ]:
# Pt of muons in the 0'th event
print(branches[["Muon_pt", "nMuon"]][0])
# All information of event 0
print(branches[0])
# For events from 10 to 14, the number of muons.
print(branches[10:15]["nMuon"])

**Exercise**: 
Print out the pt and charge of the muons from events 100 to 115.

In [ ]:
# Put your answer here
branches[100:116][["Muon_pt", "Muon_charge"]]

- In addition, you can peform the same vectorized operations we saw we could do with numpy arrays, but with Awkward arrays.

In [ ]:
muon_pt = branches["Muon_pt"][:15]
muon_pt

In [ ]:
muon_pt > 2

- Before moving on, one last thing: if we would rather use a library different than Awkward when reading from a ROOT file, we could also use Numpy or Pandas.

In [ ]:
tree["nMuon"].array(library="np", entry_stop=10)

In [ ]:
tree.arrays(library="np", entry_stop=2)

In [ ]:
tree.arrays(library="pd", entry_stop=10)

## Histograms
- Histograms are one of the most important tools in HEP. They describe the frequency distributions of a set of data points by dividing the entire range of values into a series of bins and counting how many data points fall into each interval.
- In a ROOT file, we might also find objects of type `TH1F`, which are 1D histograms. We can access these object with Uproot, but we won't be able to display the histogram with just Uproot, which is why we can use some useful methods, most of which start with "to".

In [ ]:
import matplotlib.pyplot as plt
import skhep_testdata

file_name = skhep_testdata.data_path("uproot-Event.root")
file = uproot.open(file_name)
file.keys()

In [ ]:
h = file["hstat"]
h

In [ ]:
h.to_hist().plot()

In [ ]:
h.to_numpy()

- We can also use Matplotlib, to plot the data we read from root files.

In [ ]:
# Doc: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html
plt.hist(
    branches["nMuon"], 
    bins=40
    # bins=np.arange(50)
)

plt.title("Number of Muons per Event")
plt.xlim((0, 10))
plt.ylabel("Count")
plt.xlabel("Number of muons")
plt.show()

In [ ]:
# Note that if the data has already been binned and counted, it might be better to use plt.stairs
# Doc: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.stairs.html
counts, bins = np.histogram(
    branches["nMuon"], 
    bins=40,
    # bins=np.arange(40)
)
print(counts)
print(bins)

plt.stairs(
    counts, 
    bins,
    color="red"
)
plt.title("Number of Muons per Event")
plt.xlim((0, 10))
plt.ylabel("Count")
plt.xlabel("Number of muons")
plt.show()
plt.show()

- If the histograms we are trying to make is in the form of a jagged array, we would need to flatten it first.

In [ ]:
ak.flatten(branches["Muon_pt"])


In [ ]:
plt.hist(
    ak.flatten(branches["Muon_pt"]),
    bins=100,
    range=(0,100)
)
plt.xlabel('Muon $p_{\mathrm{T}}$ [GeV]')
plt.ylabel('Number of muons / 1 GeV')
# Sometimes it might be a bit more insightful to use logarithmic axis.
# plt.yscale('log')
plt.show()

- One issue with logarithmic scales if applied to the x-axis is that the bin size is not made logarithmic as well. This produces weird looking plots like the one shown below.

In [ ]:
plt.hist(
    ak.flatten(branches["Muon_pt"]),
    bins=100,
    range=(0,100)
)
plt.xlabel('Muon $p_{\mathrm{T}}$ [GeV]')
plt.ylabel('Number of muons / 1 GeV')
# Sometimes it might be a bit more insightful to use logarithmic axis.
# plt.yscale('log')
plt.xscale("log")
plt.show()

- To fix this, we can use `np.logpsace`.

In [ ]:
plt.hist(
    ak.flatten(branches['Muon_pt']),
    bins=np.logspace(
        np.log10(1), 
        np.log10(100), 
        100
    ) # plt will use the values of this array to set the bin edges
)
plt.xlabel('Muon $p_{\mathrm{T}}$ [GeV]')
plt.xscale('log')
plt.ylabel('Number of muons')
plt.show()

**Exercise:** Make a histogram of the eta of all muons.

In [ ]:
# Place your answer here
plt.hist(ak.flatten(branches['Muon_eta']), bins=50, range=(-2.5, 2.5))
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of muons')
plt.xlim(-3, 3)
plt.show()

## Columnar Analysis
- Suppose you had an array of numbers and you wanted to check if each one of them is greater than, for instance, 25. If you are new to programming, you might think that the best way to do this might be by looping over each one of the elements. That works, but it is slow!
- Packages like numpy (on which awkward and a whole host of packages) allows us to do *vectorized* operations.


- Doing this this way is better in two primary ways:
  - Higher performance
  - Cleaner syntax
- Here's an example. Suppose we want to see how many events have above 2 muons. We can do it two ways. You be the judge of which one is better for tackling this problem!

In [ ]:
%%timeit
# Approach 1: Loops
GT2Muons = []
for nMuonsInEvent in branches["nMuon"]:
    GT2Muons.append(nMuonsInEvent > 0)

nEventsGT2Muons = sum(GT2Muons)
# print("Total number of events: ", len(GT2Muons))
# print("Total number of events with nMuon > 2: ", nEventsGT2Muons)

In [ ]:
%%timeit
# Approach 2: Vectorized operations
GT2Muons = branches["nMuon"] > 2
sum(GT2Muons)

- As you can see, the syntax is more straight-forward (it could've been done in a single line!), and the code is run *much* faster!
- A vectorized approach also allows you to easily filter out events which don't meet some criteria defined by a "mask" (i.e. array of booleans). For instance, suppose we want to plot the $\eta$ for all muons which belong to an event which has only 1 muon.

In [ ]:
single_muon_mask = branches["nMuon"] == 1
single_muon_mask

In [ ]:
muonEta_single = branches["Muon_eta"][single_muon_mask]
muonEta_single

In [ ]:
plt.hist(
    ak.flatten(muonEta_single),
    bins=100
)
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of muons')
plt.xlim(-3, 3)
plt.show()

- As another example, suppose we want to plot all of the muons which have $|\eta| < 2$. This can be done by applying the `abs` function to the array as a whole as follows.

In [ ]:
eta_mask = abs(branches["Muon_eta"]) < 2
eta_mask

In [ ]:
# Applying this mask and plotting eta
plt.hist(
    ak.flatten(branches["Muon_eta"][eta_mask]),
    bins=100
)
plt.xlabel('Muon $\eta$')
plt.ylabel('Number of muons')
plt.xlim(-3, 3)
plt.show()

- You can see that we have effectively sliced our plot such that there are zero entries beyond $|\eta| < 2$.
- Another operation which is very useful is the NOT operation. This can be applied to a whole mask all at once using the `~` operator. For instance:

In [ ]:
single_muon_mask

In [ ]:
~single_muon_mask

- You can also do the AND or OR operation between to masks. Keep in mind that the dimensions need to be the same in order for this to be possible, as the AND/OR operator requires two inputs.

In [ ]:
# AND
single_muon_mask & eta_mask

In [ ]:
# OR
single_muon_mask | eta_mask

- Keep in mind that you need to be careful about performing a series of these operations, as the order of operations might be different from what you might expect. Just keep in mind that `&` and `|` has a higher precedence than `==`. Or, you can just make things unambigous for yourself by using parenthesis around the operations which you want to be performed first. E.g.

In [ ]:
# Intended
(False == False) & False

In [ ]:
# Not what we intended
False == False & False

- Now that we know how to apply masks, lets combine this with our knowledge of histograms and compare plots when we apply differing selections.

In [ ]:
plt.hist(
    [ak.flatten(branches['Muon_pt'][single_muon_mask & eta_mask]),
    ak.flatten(branches['Muon_pt'][single_muon_mask & ~eta_mask])],
    bins=25, 
    range=(0, 50),
    histtype="step",
    density=True # Option which normalizes the integral of each historgram
)
plt.xlabel('Muon $p_{\mathrm{T}}$ [GeV]')
plt.ylabel('Number of single muons / 2 GeV')
plt.show()

## Getting Physics-Related Information
- Let's use what we have learned so far to get some physical insight into our data.
- Many mesons and bosons have a di-muon decay channel. For instance, $Z$ boson can decay into two muons.

![etadimuon](./assets/Zdimuon.png)

- In order for a process like this to happen, certain conditions must be met such as charge conservation and 4-momentum conservation. If we wanted to measure the mass of those particles which produce two muons, we could use this to our advantage by demanding that all events under consideration have, for instance, at least two muons and that those two muons have opposite charges.
- Let's start simple: we want events with exactly two muons

In [ ]:
two_muon_mask = branches["nMuon"] == 2

- Now we want to work with the 4-momentum of the muons. For this, we can use a handy library called vector which allows us to construct 4-vectors and perform operations between them, as well as compute other derived quantities from them easily.

In [ ]:
import vector

In [ ]:
muon_p4 = vector.zip(
    {
        "pt": branches["Muon_pt"],
        "eta": branches["Muon_eta"],
        "phi": branches["Muon_phi"],
        "mass": branches["Muon_mass"]
    }
)

In [ ]:
muon_p4

- We now apply the mask to keep only those events with exactly two muons.

In [ ]:
two_muons_p4 = muon_p4[two_muon_mask]
two_muons_p4

In [ ]:
type(muon_p4)

- Note that from this type of object we can obtain quantities that are derived from the inputs we gave it as we were declaring it.

In [ ]:
two_muons_p4.pt

In [ ]:
two_muons_p4.E

- Because what we want is to find the invariant mass of the two muons in each event, we sum the 4-momentum of both muons in each event together. 

In [ ]:
sum_p4 = two_muons_p4[:, 0] + two_muons_p4[:, 1]
sum_p4 # Also a 4-momentum object

- This gives us the invariant mass already.

In [ ]:
sum_p4.mass

- But not so fast... How do we know that charge conservation is not being violated? For that, we can create a mask from `branch["Muon_charge]` which ensure the charges are opposite between the two muons!

In [ ]:
# We get only those events with two muons
two_muon_events_q = branches["Muon_charge"][two_muon_mask]

# We make sure those two muons have opposite charges
opposite_sign_muons_mask = ((two_muon_events_q[:, 0] + two_muon_events_q[:, 1]) == 0)
opposite_sign_muons_mask

In [ ]:
dimuon_p4 = sum_p4[opposite_sign_muons_mask]

- We finally have our invariant masses (from dimuon events with opposite charges)! We could have done more to clean up the events, but this is already enough to get some interesting results. Lets plot a histogram of the invariant masses and see what we get!

In [ ]:
plt.hist(
    dimuon_p4.mass, 
    histtype="step", 
    bins=np.logspace(
        np.log10(0.1), 
        np.log10(1000), 
        200)
    )
plt.xscale("log")
plt.yscale("log")
plt.xlabel('Dimuon invariant mass [GeV]')
plt.ylabel('Number of dimuon events')
plt.show()

- In this plot, we can see some see peaks corresponding to particles which decay into two muons.

![dimuonspectrumplt](./assets/dimuonspectrumplt.png)

We can compute $\Delta R$ which is a measure of the distance of two particles in the detector's pseudoriapidity-azimuthal angle space. It is defined as
$$
    \Delta R = \sqrt{(\Delta \eta)^2 + (\Delta \phi)^2}
$$
If two muons are produced by the same physical process, we would expect them to be close to each other and thus have a small $\Delta R$. However, if we plot the $\Delta R$ of the muons we obtained, we get the following.

In [ ]:
two_muons_p4_oppq = two_muons_p4[opposite_sign_muons_mask]
two_muons_p4_oppq[0]

In [ ]:
plt.hist(
    two_muons_p4_oppq[:,0].deltaR(two_muons_p4_oppq[:,1]),
    bins=100,
    histtype="step"
)
plt.show()

From this plot, it is evident that our data is clearly contaminated. 

**Exercise:** Clean up the data even further and plot the mass spectrum once again.

In [ ]:
# Your answer here
close_muons = two_muons_p4_oppq[:,0].deltaR(two_muons_p4_oppq[:,1]) < 0.5
dimuon_p4_close = dimuon_p4[close_muons]

plt.hist(
    dimuon_p4_close.mass, 
    histtype="step", 
    bins=np.logspace(
        np.log10(0.1), 
        np.log10(1000), 
        200)
    )
plt.xscale("log")
plt.yscale("log")
plt.xlabel('Dimuon invariant mass [GeV]')
plt.ylabel('Number of dimuon events')
plt.show()

Answer: 
1. Can you still see each of the peaks? If no, why do you think that is? 
2. Did we suppress any of the peaks that were visible before? What does that tell us about that corresponding particle's dimuon decay channel?
3. In the $\Delta R$ plot we saw a second peak at ~3. What do you think this peak corresponds to? Plot the mass spectrum histogram again, but considering only dimuons that are far away.

In [ ]:
# Your answer here
close_muons = two_muons_p4_oppq[:,0].deltaR(two_muons_p4_oppq[:,1]) > 1.5
dimuon_p4_far = dimuon_p4[close_muons]

plt.hist(
    dimuon_p4_far.mass, 
    histtype="step", 
    bins=np.logspace(
        np.log10(0.1), 
        np.log10(1000), 
        200)
    )
plt.xscale("log")
plt.yscale("log")
plt.xlabel('Dimuon invariant mass [GeV]')
plt.ylabel('Number of dimuon events')
plt.show()